In [1]:
import jittor as jt
from jittor import nn
import math

import sys
import os

current_dir = os.path.abspath('.')
project_root = current_dir

for _ in range(2):
    project_root = os.path.dirname(project_root)
print(f"Project root determined as: {project_root}")

if project_root in sys.path:
    sys.path.remove(project_root)

sys.path.insert(0, project_root)
        

[i 1018 15:45:36.372061 84 log.cc:351] Load log_sync: 1
[i 1018 15:45:36.411550 84 compiler.py:956] Jittor(1.3.10.0) src: /home/jittor/SCC_Model/ViT/.venv/lib/python3.10/site-packages/jittor
[i 1018 15:45:36.416654 84 compiler.py:957] g++ at /usr/bin/g++(11.4.0)
[i 1018 15:45:36.417643 84 compiler.py:958] cache_path: /home/jittor/.cache/jittor/jt1.3.10/g++11.4.0/py3.10.12/Linux-6.6.87.2x4a/AMDRyzen97940Hxd7/fa38/default
[i 1018 15:45:36.489596 84 install_cuda.py:96] cuda_driver_version: [12, 9]
[i 1018 15:45:36.490744 84 install_cuda.py:82] needed restart but not /home/jittor/SCC_Model/ViT/.venv/bin/python ['-m', 'ipykernel_launcher', '--f=/mnt/wslg/runtime-dir/jupyter/runtime/kernel-v317b83fd22ffccfad1c5be6adfbe5285838de161b.json'], you can ignore this warning.
[i 1018 15:45:36.500217 84 __init__.py:412] Found /home/jittor/.cache/jittor/jtcuda/cuda12.2_cudnn8_linux/bin/nvcc(12.2.140) at /home/jittor/.cache/jittor/jtcuda/cuda12.2_cudnn8_linux/bin/nvcc.
[i 1018 15:45:36.556592 84 __init

Project root determined as: /home/jittor/SCC_Model/ViT


In [2]:
from config import Config
config = Config()

In [ ]:
class patch_embedding_Layer(nn.Module):
    def __init__(self, 
                img_size=config.IMG_SIZE, 
                in_channels=config.IN_CHANNELS, 
                patch_size=config.PATCH_SIZE, 
                embed_dim=config.EMBED_DIM):
        super(patch_embedding_Layer, self).__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = (img_size // patch_size) ** 2

        self.proj = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)
        
        # 使用可学习的参数
        self.class_token = jt.init.gauss((1, 1, embed_dim))
        self.position_embed = jt.init.gauss((1, 1 + self.num_patches, embed_dim))
        
        # 添加归一化层
        self.norm = nn.LayerNorm(embed_dim)

    def execute(self, x):
        B, C, H, W = x.shape
        assert H == self.img_size and W == self.img_size,\
            f"Input image size ({H}*{W}) doesn't match model ({self.img_size}*{self.img_size})."
        
        x = self.proj(x).view(B, self.num_patches, -1)
        
        # 添加class token
        x = jt.concat([self.class_token.expand(B, -1, -1), x], dim=1)
        
        # 添加位置编码
        x = x + self.position_embed.expand(B, -1, -1)
        
        # 添加归一化
        x = self.norm(x)
        
        return x
